In [10]:
import numpy as np
import os, sys
ROOT = os.path.abspath("..")  # or specific path
sys.path.append(os.path.join(ROOT, "src"))
import yaml
import gymnasium as gym
from stable_baselines3.common.monitor import Monitor
from gym_env.ref_env import Ref, make_env
from helpers import helper as h
from helpers import device as d

### For now, just for C05!

In [11]:
room_n = 2
c_room = 40500000
Q_dist = 33700
min_temp = -16
max_temp = -22
T_suction = -27
Q_rated = 600000
W_rated = 100000
T_rated = -25
Q_max = 2000*(T_suction - T_rated)/30 + Q_rated
W_max = -70 *(T_suction - T_rated)**2 + W_rated
high_penalty = 1
low_penalty = 0.5
delta_t = 60
bevap = 22900
e_p = 1.0e-8


In [12]:
lookahead = 3


In [13]:
T_room = np.array([-20.0, -21.0])
best_action = None
best_cost = float('inf')
    
# Generate all possible action sequences for the lookahead horizon
for action_seq in range(4**1):
    actions = [(action_seq >> (2 * i)) & 3 for i in range(1)]
    T_sim = T_room.copy()
    total_cost = 0
    print(actions)
    action_array = np.array([1 if (actions[0] >> i) & 1 else 0 for i in range(room_n)])
    print(action_array)

[0]
[0 0]
[1]
[1 0]
[2]
[0 1]
[3]
[1 1]


In [14]:
def MPC(T_room , lookahead):
    """
    Model Predictive Control for a refrigeration system with multiple rooms.
    
    Parameters:
    T_room (np.array): Current temperatures of the rooms.
    lookahead (int): Number of future time steps to consider in the optimization.
    
    Returns:
    int: Optimal action (0 or 1) for the current time step.
    """
    best_action = None
    best_cost = float('inf')
    
    # Generate all possible action sequences for the lookahead horizon
    for action_seq in range(4**lookahead):
        actions = [(action_seq >> (2 * i)) & 3 for i in range(lookahead)]
        T_sim = T_room.copy()
        total_cost = 0
        print(actions)

        
        for action in actions:
            # first map 0 - 3 to array of 0s and 1s
            action_array = np.array([1 if (action >> i) & 1 else 0 for i in range(room_n)])
            # Calculate Q_evap based on the current action
            Q_evap = bevap * (T_sim - T_suction*np.ones_like(T_sim)) * action_array
            Q_evap = np.maximum(Q_evap, 0)
            Q_total = np.minimum(np.sum(Q_evap), Q_max)
            Q_evap = Q_evap * (Q_total / (np.sum(Q_evap)))  # Scale Q_evap to not exceed Q_max
            W = W_max * ((Q_total / Q_max)*0.61 + 0.39*(Q_total>0))
            C_cost = W * e_p * (delta_t)  # Cost in dollars for the time step
            step_cost = C_cost
            # Update room temperatures
            T_sim += (delta_t / c_room) * (Q_dist - Q_evap)

            
            # Calculate cost
            
            high_violation = np.maximum(0,  T_sim - max_temp*np.ones_like(T_sim))
            low_violation = np.maximum(0,  min_temp*np.ones_like(T_sim) - T_sim)
            step_cost += (high_penalty * np.sum(high_violation) + low_penalty * np.sum(low_violation) )* delta_t
            total_cost += step_cost
            
        # Update best action if current sequence has lower cost
        if total_cost < best_cost:
            best_cost = total_cost
            best_action = actions[0]
    
    return best_action


In [16]:
C_path = 'env_config/C04.yaml'  # Default path for environment configuration
with open(f"{ROOT}/{C_path}", 'r') as f:
        env_cfg = yaml.load(f, Loader=yaml.SafeLoader)

params = env_cfg.copy()
timelimit = 1440 

In [ ]:
n_ep = 1
n_seeds = 1
reward = np.zeros (shape = (n_seeds, n_ep , timelimit))
for n in range(n_seeds):
    env = Monitor(gym.wrappers.TimeLimit(Ref(params=params), max_episode_steps=timelimit))
    for ep in range(n_ep):
        t = 0
        obs, info = env.reset(seed=n*50+ep)
        terminated, truncated = False, False
        T_room = obs["room_temperatures"]
        while not (terminated or truncated):
            t+=1
            action = MPC(T_room, lookahead)
            obs, reward[n, ep, t-1], terminated, truncated, info = env.step(action)
            T_room = obs["room_temperatures"]